# Train SISSO

In [ ]:
import os
import psutil
import json
import pandas as pd
import numpy as np
import warnings
from pathlib import Path
from mlproject.data.preprocessing import get_dataset
from mlproject.training.feature_selection import get_relevant_features, GAFeatureSelector
from sissopp.postprocess.load_models import load_model
from sissopp import FeatureSpace, SISSORegressor, Inputs
from sissopp.py_interface import get_fs_solver
warnings.filterwarnings("ignore")

In [ ]:
# Only targets where significant improvements are observed with Matminer+LOB descriptor set
target_names = ["max_pfc", "log_g_vrh", 
                "log_k_vrh", "log_klat_300", 
                "log_kp_300", "log_msd_all_300", 
                "log_msd_all_600", "log_msd_mean_600"]

In [ ]:
parent_dir = os.getcwd()

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

**Provide absolute path to https://github.com/DigiMatChem/paper-ml-with-lobster-descriptors/tree/main/data after cloning the repository locally to `data_parent_dir` variable below**

In [ ]:
data_parent_dir = "absolute/path/to/paper-ml-with-lobster-descriptors/data/"

## Approach 1: Using top 20 descriptors using feature importance scores (Quick and Cheap)

**Provide path to feature importance scores from feature reduction pipeline results i.e., Aggregated all relevant feature selection (ARFS) scores. This could be generated using the code provided [here](https://github.com/DigiMatChem/paper-ml-with-lobster-descriptors/blob/main/notebooks/ml_scripts/feature_selector/relevant_descriptors.ipynb)**

**We use top 20 descriptors as input to train SISSO models**

In [ ]:
# path to the output of the code linked above
feat_imp_dir = "absolute/path/to/arfs_descriptors"

In [ ]:
sissopp_binary_path = "/path/to/compiled/sisso++"

sissopp_inputs = {
    "data_file": "data.csv",
    "property_key": "",
    "desc_dim": 3,
    "n_sis_select": 100,
    "max_leaves": 4,
    "max_rung": 1, # Simply change value to 2 for rung 2 models
    "calc_type": "regression",
    "min_abs_feat_val": 1e-5,
    "max_abs_feat_val": 1e8,
    "n_residual": 10,
    "n_models_store": 1,
    "n_rung_generate":1,
    "n_rung_store": 0,
    "leave_out_frac": 0.0,
    "leave_out_inds": [],
    "verbose": False,
    "opset": ["add", "sub", "abs_diff", "mult", "div", "inv", "abs", "exp", "log", "sq", "cb", "sqrt", "cbrt", "neg_exp"]
}

In [ ]:
for target in target_names:
    
    # read the importance scores summary
    arfs_summ = pd.read_json(f"{feat_imp_dir}/arfs_summary_{target}.json")

    # get names of top 20 descriptors
    top20 = set(list(arfs_summ.sort_values(by="mean", ascending=False).head(20).index))

    target_df, feat_df = get_dataset(feat_type="matminer_lob", target_name=target,
        data_parent_dir=data_parent_dir)

    os.makedirs(target,exist_ok=True)

    model_input = sissopp_inputs.copy()

    model_input["property_key"] = target

    model_rung = model_input["max_rung"]

    work_dir = Path(f"{target}/rung_{model_rung}")
    work_dir.mkdir(exist_ok=True)
    os.chdir(work_dir)

    # Save input datafile
    pd.concat([feat_df.loc[:, list(top20)], target_df], axis=1).to_csv("data.csv")

    with open("sisso.json", "w") as f:
        json.dump(model_input, f, indent=4)

    inputs_base = Inputs("sisso.json")

    feature_space = FeatureSpace(inputs_base)

    sisso = SISSORegressor(inputs_base, feature_space)

    sisso.fit()

    os.chdir(parent_dir)

### Load saved models to get the equations

In [ ]:
for target in target_names:
    print(target)
    m = load_model(train_file=f"{target}/rung_1/models/train_dim_1_model_0.dat")
    print(m.latex_str.replace(",", "\_"))
    print("")

## Approach 2: GA-assisted feature selection for SISSO 

<div style="
    border-left: 5px solid #d9534f;
    background-color: #fdf2f2;
    padding: 12px;
    margin: 16px 0;
">
<strong>⚠️ Caution: Long-Running Code</strong><br><br>
The following code snippet is <strong>computationally expensive</strong> and,
in its current implementation, may take <strong>more than 24 hours</strong>
to converge for single target.<br><br>
</div>

In [ ]:
num_jobs = psutil.cpu_count(logical=False) # This will use all physical cores on the system. Please reduce it as per needs

In [ ]:
model_type = "ga_sisso"

In [ ]:
sissopp_binary_path = "/home/htc/anaik/sissopp/build/bin/sisso++"

sissopp_inputs = {
    "data_file": "data.csv",
    "property_key": "",
    "desc_dim": 3,
    "n_sis_select": 100,
    "max_leaves": 4,
    "max_rung": 2, # Descriptors for rung 2 models are optimized, if want rung 1, simply change this value to 1.
    "calc_type": "regression",
    "min_abs_feat_val": 1e-5,
    "max_abs_feat_val": 1e8,
    "n_residual": 10,
    "n_models_store": 1,
    "n_rung_generate":1,
    "n_rung_store": 0,
    "leave_out_frac": 0.0,
    "leave_out_inds": [],
    "verbose": False,
    "opset": ["add", "sub", "abs_diff", "mult", "div", "inv", "abs", "exp", "log", "sq", "cb", "sqrt", "cbrt", "neg_exp"]
}

In [ ]:
for target_name in target_names:
    for feat_type in ["matminer_lob"]:
        target, feat = get_dataset(feat_type=feat_type, target_name=target_name,
        data_parent_dir= data_parent_dir)

        os.makedirs(f"{model_type}_{target_name}_{feat_type}", exist_ok=True)
        os.chdir(f"{model_type}_{target_name}_{feat_type}")
        
        feat.dropna(axis=1, inplace=True)

        pipe, X_train_fil = get_relevant_features(
                        X_train=feat, 
                        y_train=target.values.flatten(), 
                        grootcv_n_iter=1,
                        **{"all_rel_feats__n_jobs": num_jobs})

        sissopp_inputs["property_key"] = target_name

        selector = GAFeatureSelector(
            X=X_train_fil,
            y=target.values.flatten(),
            model=None,
            scoring="neg_mean_absolute_error",
            X_test=None,
            y_test=None,
            n_jobs=psutil.cpu_count(logical=False),
            return_train_score=False,
            **{"num_features": 5,
              "sissopp_binary_path": sissopp_binary_path,
               "sissopp_inputs": sissopp_inputs,
              "mpi_tasks": num_jobs,
              "population_size": 5,
              "generations": 50}
        )

        selected_features = selector.run(strategy="de")

        with open("feature_usage_counts.json", "w") as f:
            json.dump(selector.feature_usage_counts, f)

        X_train_final = X_train_fil.loc[:, selected_features]

        pd.concat([X_train_final, target], axis=1).to_csv("data.csv")

        with open("sisso.json", "w") as f:
            json.dump(sissopp_inputs, f, indent=4)


        inputs = Inputs("sisso.json")
        _, model = get_fs_solver(inputs, allow_overwrite=False)
    
        model.fit()

        os.chdir(parent_dir)